In [232]:
import SUGen as su
import numpy as np

In [379]:
import PhysConst as pc
from scipy.io import savemat
from scipy.interpolate import interp1d

In [234]:
el = pc.PhysicsConstants()
el.numneu = 4
el.numneumax
print el.numneu

4


In [235]:
U = su.SUGen(el)

In [236]:
type(U)

SUGen.SUGen

In [237]:
a = U.matrix_gen()

In [238]:
print U.phase_gen(2,3)

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]


In [239]:
U.sample_phase()

0.730865473974113

In [252]:
U.sample_params()
B = U.matrix_gen()
print B

[[-0.43068287-0.28943282j  0.44598609+0.57192043j -0.37904617+0.2007616j
   0.13729009-0.04375564j]
 [ 0.02181808-0.41464419j  0.38361015-0.45339789j -0.07498233-0.54250151j
   0.41792990-0.01649692j]
 [-0.39993310+0.47802461j  0.04055660+0.08602876j -0.18322684-0.59868968j
  -0.24341361+0.38890694j]
 [ 0.04799601+0.40936225j -0.01414862+0.33471043j  0.17922320-0.30325927j
   0.39205874-0.66339316j]]


In [261]:
A =  B[0:3,0:3]
print abs(np.dot((A.conjugate().T),A)

[[ 0.83011894+0.j         -0.13633874-0.02185668j  0.11554090+0.08792245j]
 [-0.13633874+0.02185668j  0.88776874+0.j          0.10403980+0.05569717j]
 [ 0.11554090-0.08792245j  0.10403980-0.05569717j  0.87591286+0.j        ]]


In [242]:
rowe = A[1,:]*A[1,:]
print A*A
print rowe

[[ 0.40295778  0.32504171  0.04643977  0.22556074]
 [ 0.05579645  0.321307    0.26514957  0.35774698]
 [ 0.00215232  0.22511435  0.68337792  0.0893554 ]
 [ 0.53909345  0.12853693  0.00503274  0.32733688]]
[ 0.05579645  0.321307    0.26514957  0.35774698]


In [243]:
init_prod = np.asarray([[1,2,0],[1,0,0],[0,1,0],[0,0,1]])
init_prod[1,:]*[2,3,0]

array([2, 0, 0])

In [244]:
a = [[1,2,3],[2,3,4],[5,3,4]]
b = np.asarray(a)
print b[1,:]

[2 3 4]


In [245]:
b = A[2,:]

In [305]:
def fractionFromUnitaryMatrix(A):
    init_prod = np.asarray([[1.0/3,2.0/3,0],[1,0,0],[0,1,0],[0,0,1]])
    
    rowe = A[0,:]*A[0,:]
    rowmu = A[1,:]*A[1,:]
    rowtau = A[2,:]*A[2,:]
    mat2 = A*A
    termE = []
    termMu = []
    termTau = []
    for i in range(3):
        termE.append(sum(mat2[i,:]*rowe))
        termMu.append(sum(mat2[i,:]*rowmu))
        termTau.append(sum(mat2[i,:]*rowtau))
    
    
    #implementing this for testing purposes
    final_e = []
    final_mu = []
    final_tau = []

    for i in range(4):
        final_e.append(sum(init_prod[i,:]*termE))
        final_mu.append(sum(init_prod[i,:]*termMu))
        final_tau.append(sum(init_prod[i,:]*termTau))
    
    
    total = np.add(final_e, final_mu)
    total = np.add(total, final_tau)
    return final_e, final_mu, final_tau, total
        
    

In [385]:
def processFile(filename):
    csv = np.genfromtxt(filename, delimiter=",")
    csv = csv[np.argsort(csv[:, 0])]
    #print csv.shape
    x = csv[:,0]
    index = []
    for i in range(len(x) - 1):
        if x[i] == x[i+1]:
            index.append(i)
    #index = index
    for i in reversed(index):
        csv = np.delete(csv,i,0)
    x = csv[:,0]
    y = csv[:,1]
    #print x.shape, y.shape
    f = interp1d(x,y)
    #f = interpld(x,y)
    return f

In [386]:
def dagger(U):
    return np.dot((U.conjugate().T),U)

In [389]:
def checkForCuts(mat,interpolators):
    m = mat.flatten()
    count = 0
    totalChi2 = 0
    chi2Limit = 10.42
    for i in m:
        try:
            temp = interpolators[count](i)
            if temp < -1:
                print temp, i, count
        except ValueError:
            return False
        count = count + 1
        totalChi2 = totalChi2 + temp
    
    if totalChi2 <= chi2Limit:
        print totalChi2
        return True
    else:
        return False
            
        
    

In [390]:
count = 0
frac_e = []
frac_mu = []
frac_tau = []
phyEl = pc.PhysicsConstants()
phyEl.numneu = 4
ndim = 3
count  = 0
U = su.SUGen(phyEl)
non_unitarity_frac = 0.3
interpolators = []
interpolators.append(processFile('ue1.csv'))
interpolators.append(processFile('ue2.csv'))
interpolators.append(processFile('ue3.csv'))
interpolators.append(processFile('umu1.csv'))
interpolators.append(processFile('umu2.csv'))
interpolators.append(processFile('umu3.csv'))
interpolators.append(processFile('utau1.csv'))
interpolators.append(processFile('utau2.csv'))
interpolators.append(processFile('utau3.csv'))
# fe2 = processFile('ue2.csv')
# fe3 = processFile('ue3.csv')
# fmu1 = processFile('umu1.csv')
# fmu2 = processFile('umu2.csv')
# fmu3 = processFile('umu3.csv')
# ftau1 = processFile('utau1.csv')
# ftau2 = processFile('utau2.csv')
# ftau3 = processFile('utau3.csv')
while True:
    U.sample_params()
    matrix = (U.matrix_gen())
    mat33 = matrix[0:3,0:3]
    
    I = np.identity(ndim)
    matDag = dagger(mat33)
    delta = np.subtract(matDag,I)
#     if count < 10:
#         print np.absolute(delta)
#         print mat33
#         count = count + 1
    if np.amax(np.absolute(delta)) <= non_unitarity_frac :
        flag = checkForCuts(np.absolute(mat33),interpolators)
        if flag == True:
            print "found one " + str(count)
            count = count + 1
            efrac, mufrac, taufrac,total = fractionFromUnitaryMatrix(np.absolute(mat33))
    #         frac_e.append(np.divide(efrac,total))
    #         frac_mu.append(np.divide(mufrac,total))
    #         frac_tau.append(np.divide(taufrac,total))
            frac_e.append(efrac)
            frac_mu.append(mufrac)
            frac_tau.append(taufrac)
        #print efrac, total
    
    if count > 100:
        break

frac_e = np.asarray(frac_e)
frac_mu = np.asarray(frac_mu)
frac_tau = np.asarray(frac_tau)
#savemat('frac_30.mat',mdict={'eFraction' : frac_e, 'muFrac':frac_mu, 'tauFrac':frac_tau})
    
    

5.28269312463
found one 0
8.67652937294
found one 1
7.80516721687
found one 2
9.60871355342
found one 3
10.1385381572
found one 4
9.45567436465
found one 5
6.78980794481
found one 6
3.58616465613
found one 7
6.82334903084
found one 8
9.59068778264
found one 9
4.18354965938
found one 10
7.75364991477
found one 11
8.95504488654
found one 12
8.88373490369
found one 13
3.95631332205
found one 14
5.17423810665
found one 15
3.70063586412
found one 16
7.14943576964
found one 17
5.61372094883
found one 18
1.91197937097
found one 19
4.39123583036
found one 20
8.41151923271
found one 21
8.24000123482
found one 22
6.96438266498
found one 23
4.85548767776
found one 24
9.11551386471
found one 25
5.31781188387
found one 26
3.569531233
found one 27
9.41004154624
found one 28
6.31683522606
found one 29
7.95531850022
found one 30
6.25493203389
found one 31
5.21062119282
found one 32
7.81411500506
found one 33
1.91868816469
found one 34
10.2765790046
found one 35
4.73966376363
found one 36
4.20850458912

In [361]:
m = np.array([[1,2,3],[4,5,6],[12,34,35]])
for i in m.flatten():
    print i
    

1
2
3
4
5
6
12
34
35


In [371]:
fe1 = processFile('ue1.csv')
fe2 = processFile('ue2.csv')
fe3 = processFile('ue3.csv')
fmu1 = processFile('umu1.csv')
fmu2 = processFile('umu2.csv')
fmu3 = processFile('umu3.csv')
ftau1 = processFile('utau1.csv')
ftau2 = processFile('utau2.csv')
ftau3 = processFile('utau3.csv')

In [374]:
fe1(0.802)

array(-0.3534561629474408)

In [278]:
phyEl = pc.PhysicsConstants()
phyEl.numneu = 4
U = su.SUGen(phyEl)
U.sample_params()
matrix = np.absolute(U.matrix_gen())
A = matrix[0:3,0:3]
#print matrix
a,b,c,d = fractionFromUnitaryMatrix(A)
print d
B =  np.asarray([[0.8425,    0.5181 ,   0.1471],
   [-0.4879 ,   0.6184,    0.6160],
    [0.2282 ,  -0.5908  ,  0.7739]])
a,b,c,d = fractionFromUnitaryMatrix(B)
print d

[[ 0.27193407  0.80026217  0.35453318]
 [ 0.28082918  0.49068004  0.74276502]
 [ 0.79960899  0.21039744  0.2486023 ]]
[ 2.13039025  0.74417176  0.69310925  0.59315643]
[ 2.99952808  0.99979212  0.99986798  1.00000858]


In [223]:
a = np.identity(4)
a[3,3] = 0
print a

[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]]


In [209]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.identity(3)
print x1
print x2
np.subtract(x1, x2)


[[ 0.  1.  2.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


array([[-1.,  1.,  2.],
       [ 3.,  3.,  5.],
       [ 6.,  7.,  7.]])